<a href="https://colab.research.google.com/github/eflores5000/08MIAR_APR/blob/main/proyecto_final_APR_V06_x_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
#drive.flush_and_unmount()
#drive.mount('/content/drive')

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models', 'checkpoints', 'dqn_05_weights.h5f.data-00000-of-00001', 'dqn_05_weights.h5f.index', 'dqn_06_weights.h5f.index', 'dqn_06_weights.h5f.data-00000-of-00001', 'dqn_07_weights.h5f.index', 'dqn_07_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.index', '.ipynb_checkpoints', 'dqn_09_weights.h5f.index', 'dqn_09_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.index', 'video', 'dqn_06_3_weights.h5f.index', 'dqn_06

## 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.05
  %pip install Keras==2.2.4
  %pip install tensorflow==2.1.0
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-59j7que2
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-59j7que2
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [ ]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

#--- nuevo -- ejfm
height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## 1. EJFM Implementación de la red neuronal

In [ ]:
def build_model(height, width, channels, actions):

    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(4, 84, 84)))  # WINDOW_LENGTH, H, W
    model.add(Conv2D(32, (8, 8), strides=(4, 4),padding='SAME', activation='relu')) #, input_shape=(4, 84, 84), data_format='channels_first')) #input_shape=(4,84,84,1)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2),padding='SAME', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1),padding='SAME', activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

In [ ]:
model = build_model(84, 84, 1, nb_actions)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 32)        8224      
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 7744)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               3965440   
                                                                 
 dense_4 (Dense)             (None, 6)                

## 2. EJFM Implementación de la solución DQN

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),
        attr='eps',
        value_max=1.0,
        value_min=0.1,
        value_test=0.05,
        nb_steps=1000000)

    memory = SequentialMemory(
        limit=200000,                       # Replay Buffer (Memoria de experiencias pasadas (s,a,r,s'))
        window_length=4)

    dqn = DQNAgent(model=model,
                   nb_actions=nb_actions,
                   policy=policy,
                   memory=memory,
                   processor=AtariProcessor(),
                   nb_steps_warmup=50000,
                   gamma=0.99,
                   target_model_update=10000, # cada 10,000 steps actualiza la red objetivo
                   train_interval=4,          # Solo entrena cada n pasos de interacción con el entorno
                   delta_clip=1.0,
                   enable_double_dqn=True,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   batch_size=32,
                   )
    return dqn

In [ ]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 10,000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v06_x_weights_{step}.h5f',
    interval=10000,
    verbose=1
)

In [ ]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [ ]:
from keras.optimizers import Adam

dqn = build_agent(model, nb_actions)
dqn.compile(Adam(learning_rate=0.00025, epsilon=0.01))


In [23]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=300000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_v06_x_weights.h5f')


⏱ Tiempo de inicio: 03:05:46
Training for 300000 steps ...
    706/300000: episode: 1, duration: 3.587s, episode steps: 706, steps per second: 197, episode reward: 11.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.511 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1436/300000: episode: 2, duration: 4.474s, episode steps: 730, steps per second: 163, episode reward: 10.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.377 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2238/300000: episode: 3, duration: 3.762s, episode steps: 802, steps per second: 213, episode reward: 12.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.455 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3191/300000: episode: 4, duration: 4.498s, episode steps: 953, steps per second: 212, episode reward: 15.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.489 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   4007/300000: episode: 5, duration: 5.034s, episode

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  50468/300000: episode: 71, duration: 32.574s, episode steps: 651, steps per second:  20, episode reward:  9.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.605 [0.000, 5.000],  loss: 0.005460, mean_q: 0.049717, mean_eps: 0.954789
  51571/300000: episode: 72, duration: 75.297s, episode steps: 1103, steps per second:  15, episode reward: 17.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.510 [0.000, 5.000],  loss: 0.007758, mean_q: 0.055053, mean_eps: 0.954084
  52187/300000: episode: 73, duration: 43.540s, episode steps: 616, steps per second:  14, episode reward: 10.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.498 [0.000, 5.000],  loss: 0.007586, mean_q: 0.042686, mean_eps: 0.953310
  52847/300000: episode: 74, duration: 45.868s, episode steps: 660, steps per second:  14, episode reward:  9.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.497 [0.000, 5.000],  loss: 0.007282, mean_q: 0.047863, mean_eps: 0.952736
  54547/300000: episode: 75, du

In [24]:
# Testing part to calculate the mean reward a los 500k
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_v06_x_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 10.000, steps: 808
Episode 2: reward: 11.000, steps: 836
Episode 3: reward: 11.000, steps: 816
Episode 4: reward: 13.000, steps: 864
Episode 5: reward: 11.000, steps: 849
Episode 6: reward: 6.000, steps: 457
Episode 7: reward: 18.000, steps: 1214
Episode 8: reward: 12.000, steps: 807
Episode 9: reward: 11.000, steps: 804
Episode 10: reward: 10.000, steps: 845


In [26]:
# Se entrena 300k pasos adicionales

# Cargar los pesos desde el checkpoint
dqn.load_weights("checkpoints/dqn_v06_x_weights_300000.h5f")

# Continuar entrenamiento
from datetime import datetime
print(f"⏱ Inicio nuevo entrenamiento: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=300000, visualize=False, verbose=2, callbacks=[checkpoint_callback])

print(f"⏱ Fin nuevo entrenamiento: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_v06_x2_weights.h5f')

⏱ Inicio nuevo entrenamiento: 09:50:09
Training for 300000 steps ...
    709/300000: episode: 1, duration: 3.308s, episode steps: 709, steps per second: 214, episode reward: 11.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.557 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1422/300000: episode: 2, duration: 3.513s, episode steps: 713, steps per second: 203, episode reward: 12.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.597 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   1815/300000: episode: 3, duration: 3.204s, episode steps: 393, steps per second: 123, episode reward:  2.000, mean reward:  0.005 [ 0.000,  1.000], mean action: 2.539 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   2352/300000: episode: 4, duration: 2.596s, episode steps: 537, steps per second: 207, episode reward:  3.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.579 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   3247/300000: episode: 5, duration: 4.192

In [27]:
# Testing part to calculate the mean reward a los 500k
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_v06_x2_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 6.000, steps: 553
Episode 2: reward: 17.000, steps: 773
Episode 3: reward: 8.000, steps: 629
Episode 4: reward: 14.000, steps: 810
Episode 5: reward: 12.000, steps: 1004
Episode 6: reward: 16.000, steps: 735
Episode 7: reward: 11.000, steps: 664
Episode 8: reward: 9.000, steps: 640
Episode 9: reward: 6.000, steps: 849
Episode 10: reward: 12.000, steps: 641


In [29]:
from google.colab import drive
import shutil
import glob  # <-- ¡Falta este import!
import os

# 1. Montar Google Drive
#drive.mount('/content/drive')

# 2. Crear directorio en Drive si no existe
os.makedirs('/content/gdrive/MyDrive/Colab_Outputs', exist_ok=True)

# 3. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('dqn_v06_x_weights*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )
    print(f"¡Archivo {archivo} copiado a Drive!")

# 4. Buscar y copiar todos los archivos coincidentes
archivos = glob.glob('checkpoints/dqn_v06_x_weights_570000*')  # Encuentra todos los archivos con ese patrón

for archivo in archivos:
    shutil.copy(
        archivo,
        f'/content/gdrive/MyDrive/Colab_Outputs/{os.path.basename(archivo)}'
    )

print("✅ ¡Todos los archivos copiados!")

¡Archivo dqn_v06_x_weights.h5f.data-00000-of-00001 copiado a Drive!
¡Archivo dqn_v06_x_weights.h5f.index copiado a Drive!
✅ ¡Todos los archivos copiados!
